## <span style="color:blue;">A. Build a baseline model </span>

In [75]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

concrete_data = pd.read_csv('data/concrete_data.csv')
target = concrete_data['Strength']
predictors = concrete_data[concrete_data.columns[concrete_data.columns != 'Strength']]
# define regression model with 1 hidden layer with 10 neurons and relu activation function
def regression_model():
    # create model
    model = Sequential() # Sequential model is a linear stack of layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) # 10 neurons in the first hidden layer
    model.add(Dense(1)) # output layer with 1 neuron
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error') # adam is a popular optimizer for deep learning vs gradient descent
    return model

#### 1. Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [76]:

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)   
print("number of test samples :", X_test.shape[0])
print("number of training samples:",X_train.shape[0])
n_cols = X_train.shape[1] # number of predictors

number of test samples : 309
number of training samples: 721


#### 2. Train the model on the training data using 50 epochs.

In [77]:
model = regression_model()
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)

/Users/mark/anaconda3/envs/dsfs/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [78]:
loss_val = model.evaluate(X_test, y_test) # loss value is the MSE since we defined in the model above

y_test_ = model.predict(X_test)
y_test_ = y_test_.reshape(-1) 
from sklearn.metrics import mean_squared_error # Suggested in the assignment
mse = np.mean((y_test_ - y_test) ** 2)

# The following are identical 
print("Mean Squared Error: {}".format(mse))
print("Loss Value: {}".format(loss_val))

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 394us/step - loss: 147.3889
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Mean Squared Error: 139.06853548155794
Loss Value: 139.06854248046875


#### 4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [79]:
mse_list = []
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    y_test_ = model.predict(X_test)
    y_test_ = y_test_.reshape(-1)
    mse = mean_squared_error(y_test, y_test_)
    mse_list.append(mse)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 314us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step
10/10 ━━━━━━━━━━

#### 5. Report the mean and the standard deviation of the mean squared errors.

In [80]:
stepA_mean_mse = np.mean(mse_list)
stepA_std_mse = np.std(mse_list)
print("Mean of MSE: {}".format(stepA_mean_mse))
print("Standard Deviation of MSE: {}".format(stepA_std_mse))

Mean of MSE: 114.01276758556071
Standard Deviation of MSE: 9.742199356520802


## <span style="color:blue;">B. Normalize the data</span>

In [81]:
# Normalize the predictors by substracting the mean and dividing by the standard deviation
# Python thing: any math operation on a dataframe is applied to all the elements
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [82]:
mse_list = []
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    y_test_ = model.predict(X_test)
    y_test_ = y_test_.reshape(-1)
    mse = mean_squared_error(y_test, y_test_)
    mse_list.append(mse)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step
10/10 ━━━━━━━━━━

In [83]:
stepB_mean_mse = np.mean(mse_list)
stepB_std_mse  = np.std(mse_list)
print("Mean of MSE from Step A: {}, from Step B with normalized data: {}".format(stepA_mean_mse, stepB_mean_mse))
print("Standard Deviation of MSE from Step A: {}, from Step B with normalized data: {}".format(stepA_std_mse, stepB_std_mse))

Mean of MSE from Step A: 114.01276758556071, from Step B with normalized data: 60.98791564237656
Standard Deviation of MSE from Step A: 9.742199356520802, from Step B with normalized data: 40.20574219021084


## <span style="color:blue;">C. Increase the number of epochs</span>

In [84]:
mse_list = []
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    y_test_ = model.predict(X_test)
    y_test_ = y_test_.reshape(-1)
    mse = mean_squared_error(y_test, y_test_)
    mse_list.append(mse)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 344us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 329us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step
10/10 ━━━━━━━━━━

In [85]:
stepC_mean_mse = np.mean(mse_list)
stepC_std_mse  = np.std(mse_list)
print("Mean of MSE from Step B: {}, from Step C with 100 epochs: {}".format(stepB_mean_mse, stepC_mean_mse))
print("Standard Deviation of MSE from Step B: {}, Step C with 100 epochs: {}".format(stepB_std_mse, stepC_std_mse))

Mean of MSE from Step B: 60.98791564237656, from Step C with 100 epochs: 38.691029807292296
Standard Deviation of MSE from Step B: 40.20574219021084, Step C with 100 epochs: 3.3013260575780254


## <span style="color:blue;">

## <span style="color:blue;">D. Increase the number of hidden layers</span>

In [86]:
# define regression model with 3 hidden layers
def regression_model():
    # create model
    model = Sequential() # Sequential model is a linear stack of layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) # 10 neurons in the first hidden layer
    model.add(Dense(10, activation='relu')) # 10 neurons in the second hidden layer
    model.add(Dense(10, activation='relu')) # 10 neurons in the third hidden layer
    model.add(Dense(1)) # output layer with 1 neuron
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error') # adam is a popular optimizer for deep learning vs gradient descent
    return model

In [87]:
model = regression_model()
mse_list = []
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    y_test_ = model.predict(X_test)
    y_test_ = y_test_.reshape(-1)
    mse = mean_squared_error(y_test, y_test_)
    mse_list.append(mse)


/Users/mark/anaconda3/envs/dsfs/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 449us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step
10/10 ━━━━━━━━━━━

In [88]:
stepD_mean_mse = np.mean(mse_list)
stepD_std_mse  = np.std(mse_list)
print("Mean of MSE from Step B: {}, from Step D with three hidden layers: {}".format(stepB_mean_mse, stepD_mean_mse))
print("Standard Deviation of MSE from Step B: {}, from Step D with 3 hidden : {}".format(stepB_std_mse, stepD_std_mse))

Mean of MSE from Step B: 60.98791564237656, from Step D with three hidden layers: 36.423246046418086
Standard Deviation of MSE from Step B: 40.20574219021084, from Step D with 3 hidden : 26.183634357379013
